In [1]:
%matplotlib inline
from pynwb import NWBHDF5IO
import numpy as np 
import matplotlib.pyplot as plt
from dandi.dandiapi import DandiAPIClient
from compress_multiphoton import compute_sensitivity
from fsspec.implementations.cached import CachingFileSystem
import pathlib
import fsspec
import pynwb
import h5py
import colorcet as cc

In [2]:
def make_figure(scan, figure_filename, title=None):
    qs = compute_sensitivity(scan.transpose(1,2,0))
    print('Quantal size: {sensitivity}\nIntercept: {zero_level}\n'.format(**qs))
    
    fig, axx = plt.subplots(2,2, figsize=(9, 7))
    q = qs['sensitivity']
    b = qs['zero_level']
    axx = iter(axx.flatten())

    ax = next(axx)
    m = scan.mean(axis=0)
    _ = ax.imshow(m, vmin=0, vmax=np.quantile(m, 0.999), cmap='gray')
    ax.axis(False)
    ax.set_title('average')

    ax = next(axx)
    v = ((scan[1:,:,:].astype('float64') - scan[:-1,:,:]) ** 2/2).mean(axis=0)
    imx = np.stack(((m-b)/q, v/q/q, (m-b)/q), axis=-1)
    _ = ax.imshow(np.minimum(1, np.sqrt(0.01 + np.maximum(0, imx/np.quantile(imx, 0.9999))) - 0.1), cmap='PiYG')
    cbar = plt.colorbar(_, ax=ax, ticks=[0.05, .5, 0.95])
    cbar.ax.set_yticklabels(['<< 1', '1', '>> 1'])  
    ax.axis(False)
    ax.set_title('coefficient of variation')

    ax = next(axx)
    x = np.arange(qs["min_intensity"], qs["max_intensity"])
    fit = qs["model"].predict(x.reshape(-1, 1))
    ax.scatter(x, np.minimum(fit[-1]*2, qs["variance"]), s=2, alpha=0.5)
    ax.plot(x, fit, 'r')
    ax.grid(True)
    ax.set
    ax.set_xlabel('intensity')
    ax.set_ylabel('variance')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    ax = next(axx)
    im = (scan.mean(axis=0)-qs['zero_level'])/qs['sensitivity']
    mx = np.quantile(im, 0.999)
    _ = ax.imshow(im, vmin=-mx, vmax=mx, cmap=cc.cm.CET_D13)
    plt.colorbar(_, ax=ax)
    ax.axis(False)
    ax.set_title('Quantum flux\nphotons / pixel / frame');

    plt.suptitle(f'{title or figure_filename}\nPhoton sensitivity: {qs["sensitivity"]:4.1f}')
    fig.savefig(figure_filename, dpi=300)

In [3]:
# get all paths for microns dataset latest published version
dandiset_id = "000402"   # MICRONS
dandiset_id = "000037"   # OpenScope

with DandiAPIClient() as client:
    assets = client.get_dandiset(dandiset_id).get_assets()
    s3_urls = [x.get_content_url(follow_redirects=1, strip_query=True) for x in assets]

In [4]:
# create a caching scheme for DANDI downloads
cache_path = pathlib.Path('./cache');
cache_path.mkdir(parents=True, exist_ok=True)
fs = CachingFileSystem(fs=fsspec.filesystem("http"), cache_storage=str(cache_path))

In [ ]:
# make figures for file collections on DANDI
figure_path = pathlib.Path('./figures') / f"dandi-{dandiset_id}"
figure_path.mkdir(parents=True, exist_ok=True)

for url in s3_urls:
    # open the file
    with fs.open(url, "rb") as f:
        with h5py.File(f) as file:
            with pynwb.NWBHDF5IO(file=file, load_namespaces=True) as io:

                # get all two-photon series objects
                collection = (  
                    _ for _ in io.read().objects.values() 
                    if isinstance(_, pynwb.ophys.TwoPhotonSeries))

                for count, two_photon_series in enumerate(collection):
                    
                    if count <= 4:
                        continue

                    # dx, dy = two_photon_series.imaging_plane.grid_spacing[:]
                    # timestamps = two_photon_series.timestamps[:]
                    # for some datasets, might have to use two_photon_series.rate

                    scan = two_photon_series.data[250:750]
                    print('::-::')

                    try:
                        make_figure(scan, figure_path / f"{url.split('/')[-1]}-{count:03}.png", 
                                title=f"NWB-id:{two_photon_series.get_ancestor().identifier}\n{two_photon_series.get_ancestor().session_id}")
                    except Exception as e:
                        print(e)

/Users/dimitri/opt/miniconda3/envs/benv/lib/python3.10/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/dimitri/opt/miniconda3/envs/benv/lib/python3.10/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/dimitri/opt/miniconda3/envs/benv/lib/python3.10/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.4.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
